In [1]:
corpus = [
    "東京タワーへのアクセス。",
    "どんなお土産がありますか",
    "私は温泉に行きたいです",
    "東急ホテル予約したい",
    "浅草寺への行き方",
    "ご飯を食べたいです。",
    "私はタクシーを呼びたい",
    "博物館のチケットを買いたいです",
    "新宿までの距離は？",
    "周辺にはどんなレストランがありますか？"
    
]

## TF/IDF

In [2]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import MeCab
mecab = MeCab.Tagger ("-Owakati")

In [3]:
for index,sentence in enumerate(corpus):
    corpus[index] = mecab.parse(sentence)

In [4]:
corpus

['東京 タワー へ の アクセス 。 \n',
 'どんな お 土産 が あり ます か \n',
 '私 は 温泉 に 行き たい です \n',
 '東急 ホテル 予約 し たい \n',
 '浅草寺 へ の 行き 方 \n',
 'ご飯 を 食べ たい です 。 \n',
 '私 は タクシー を 呼び たい \n',
 '博物館 の チケット を 買い たい です \n',
 '新宿 まで の 距離 は ？ \n',
 '周辺 に は どんな レストラン が あり ます か ？ \n']

In [5]:
NG_filter = ["です","ます","？","。"]

for index,sentence in enumerate(corpus):
    word_list = sentence.split(" ")
    for word in word_list:
        if word in NG_filter:
            word_list.remove(word)
    corpus[index] = " ".join(word_list)

In [6]:
corpus

['東京 タワー へ の アクセス \n',
 'どんな お 土産 が あり か \n',
 '私 は 温泉 に 行き たい \n',
 '東急 ホテル 予約 し たい \n',
 '浅草寺 へ の 行き 方 \n',
 'ご飯 を 食べ たい 。 \n',
 '私 は タクシー を 呼び たい \n',
 '博物館 の チケット を 買い たい \n',
 '新宿 まで の 距離 は \n',
 '周辺 に は どんな レストラン が あり か \n']

In [7]:
vectorizer = TfidfVectorizer()

In [8]:
tfidf_X = vectorizer.fit_transform(corpus).toarray()

In [9]:
index = tfidf_X.argsort(axis=1)[:,::-1]
feature_names = np.array(vectorizer.get_feature_names())
feature_words = feature_names[index]

In [10]:
feature_names

array(['あり', 'ご飯', 'たい', 'どんな', 'まで', 'アクセス', 'タクシー', 'タワー', 'チケット',
       'ホテル', 'レストラン', '予約', '博物館', '周辺', '呼び', '土産', '新宿', '東京', '東急',
       '浅草寺', '温泉', '行き', '買い', '距離', '食べ'], dtype='<U5')

In [11]:
feature_words.shape

(10, 25)

In [12]:
n = 3  # top何単語取るか

for sentence in feature_words:
    print(sentence[:3])

['アクセス' '東京' 'タワー']
['土産' 'あり' 'どんな']
['温泉' '行き' 'たい']
['予約' '東急' 'ホテル']
['浅草寺' '行き' '食べ']
['食べ' 'ご飯' 'たい']
['タクシー' '呼び' 'たい']
['博物館' '買い' 'チケット']
['まで' '新宿' '距離']
['周辺' 'レストラン' 'あり']


↓  
重要度の高い単語を抽出できた

## LDA

In [13]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

In [14]:
topic_num=3 #トピック数は事前に考えないといけない
lda = LDA(n_components=topic_num, max_iter=30, n_jobs=-1)

In [15]:
lda.fit(tfidf_X)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=30, mean_change_tol=0.001,
             n_components=3, n_jobs=-1, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [16]:
 for i, component in enumerate(lda.components_[:topic_num]):
        print("topic:", i)
        idx = component.argsort()[::-1][:3]
        for j in idx:
            print(feature_names[j], component[j])

topic: 0
たい 1.4685267036020175
温泉 1.023094024248129
土産 0.9683873029077801
topic: 1
食べ 0.9823756240277501
ご飯 0.9823756240277501
新宿 0.9083906686523513
topic: 2
浅草寺 1.090480955708724
タクシー 0.9800907789852604
呼び 0.9800907789852604


↓  
データ数が少ないから、連動性見つけず、結果はあまり良くない。さらに、LDAの欠点としてはtopicの意味が明示されてない

## LDA prediction test

In [17]:
text1="ステーキを食べたい"


test1=[text1]

for index,sentence in enumerate(test1):
    test1[index] = mecab.parse(sentence)
    

X_test1 = vectorizer.transform(test1)
lda_test1 = lda.transform(X_test1)

for index,score in enumerate(lda_test1):
    print("### ",index)
    topicid = np.argmax(score)
    print(text1)
    print(score," >>> topic",topicid)
    print("")

###  0
ステーキを食べたい
[0.17602663 0.67708717 0.1468862 ]  >>> topic 1



↓  
食事系として予測されてますが、まあまあ。。